# 구조화된 출력 (Structured Output)

**텍스트를 라벨로 분류하기**

**태깅(Tagging)**이란 문서의 종류를 분류하여 **라벨링(Labeling)**하는 것을 의미합니다: 예) 긍정적, 부정적, 중립적

![이미지 설명](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/tagging.png?raw=1)

---

## **개요 (Overview)**  

**태깅(Tagging)**은 몇 가지 주요 구성 요소로 이루어집니다:  

- **`function`**: 추출(Extraction)과 마찬가지로, 태깅은 함수(Functions)를 사용하여 모델이 문서를 어떻게 태깅해야 하는지 명시합니다.  
- **`schema`**: 문서를 어떻게 태깅할지 정의합니다 --> Pydantic 데이터 모델을 이용하여 정의

In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

## Schema 정의와 도구 호출 **  

LangChain에서 OpenAI의 **도구 호출(Tool Calling)** 기능을 사용하여 태깅을 수행하는 간단한 예제를 살펴보겠습니다.  

- OpenAI 모델에서 지원하는 `with_structured_output` 메서드를 사용할 것입니다.  

스키마에 몇 가지 속성과 예상 유형을 추가하여 Pydantic 모델을 지정해 보겠습니다.

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# 프롬프트 템플릿 정의
# 주어진 텍스트에서 필요한 정보를 추출하도록 지침 제공
tagging_prompt = ChatPromptTemplate.from_template(
"""
다음 글에서 원하는 정보를 추출하세요.
'Classification' 함수에 언급된 속성만 추출하세요.

글:
{input}
"""
)

# Pydantic 데이터 모델을 이용하여 텍스트에서 추출할 속성 정의
class Classification_1(BaseModel):
    sentiment: str = Field(description="텍스트의 감정")
    agressiveness: int = Field(
        description="텍스트가 1~10점 척도로 얼마나 공격적인지를 나타냅니다."
    )

# OpenAI GPT-4o-mini 모델을 사용하여 Structured Output(구조화된 출력) 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(Classification_1)

In [14]:
inp = "너를 만나게 되어 정말 기뻐! 우리는 아주 좋은 친구가 될 것 같아!"

prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

print(response)
response

sentiment='기쁨' agressiveness=1


Classification_1(sentiment='기쁨', agressiveness=1)

사전 출력을 원하면 `.model_dump()`를 호출하면 됩니다.

In [15]:
response.model_dump()

{'sentiment': '기쁨', 'agressiveness': 1}

예제에서 볼 수 있듯이, 모델은 우리가 원하는 바를 정확하게 해석합니다.  

다음 섹션에서는 이러한 결과를 어떻게 제어할 수 있는지 살펴보겠습니다.

## **더 세밀한 출력 제어**

**스키마(schema)**를 더 자세히 정의하면 모델의 출력을 더 세밀하게 제어할 수 있습니다.  

구체적으로 다음을 정의할 수 있습니다:  

- **각 속성의 가능한 값**  
- **속성을 모델이 정확하게 이해할 수 있도록 설명 추가**  
- **반드시 반환해야 할 필수 속성**  

이전에 언급한 각 요소를 제어하기 위해 **Enums**를 사용하여 우리의 **Pydantic 모델**을 다시 선언해봅시다.



In [17]:
class Classification_2(BaseModel):
    sentiment: str = Field(..., enum=["행복하다", "중립적", "슬프다"])
    aggressiveness: int = Field(
        ...,
        description="문장이 얼마나 공격적인지를 나타내며 숫자가 높을수록 더 공격적입니다.",
        enum=[1, 2, 3, 4, 5],
    )

# OpenAI GPT-4o-mini 모델을 사용하여 Structured Output(구조화된 출력) 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(Classification_2)

이제 답변은 우리가 예상하는 방식으로 제한될 것입니다!

In [18]:
inp = "너를 만나게 되어 정말 기뻐! 우리는 아주 좋은 친구가 될 것 같아!"

prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

print(response)
response

sentiment='행복하다' aggressiveness=1


Classification_2(sentiment='행복하다', aggressiveness=1)

In [19]:
inp = "너에게 정말 화가 나! 제대로 혼내줄 거야!"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification_2(sentiment='슬프다', aggressiveness=5)

In [20]:
inp = "여기 날씨는 괜찮아요, 코트 하나 없이도 밖에 나갈 수 있어요"
prompt = tagging_prompt.invoke({"input": inp})
llm.invoke(prompt)

Classification_2(sentiment='중립적', aggressiveness=1)

### **구조화된 출력 (Structured Outputs)과 함수 호출**
 
챗봇과 같은 많은 애플리케이션에서는 모델이 사용자에게 **자연어로 직접 응답**해야 합니다. 그러나 경우에 따라 모델이 **구조화된 형식(structured format)**으로 출력을 제공해야 할 필요가 있습니다.  

예를 들어, 모델의 출력을 **데이터베이스에 저장**해야 하는 상황에서 출력이 데이터베이스 **스키마(schema)**에 맞도록 보장해야 할 수 있습니다.  

이러한 필요성은 **구조화된 출력(Structured Output)** 개념을 부각시키며, 이를 통해 모델이 **특정 출력 구조**를 따르도록 지시할 수 있습니다.  

**핵심 포인트:**  
- 자연어 응답이 아닌 **구조화된 데이터**로 응답.  
- 데이터베이스, API 등과의 호환성을 보장.  
- 스키마를 따르는 일관된 형식으로 데이터 제공.

구조화된 출력은 특히 **데이터베이스 저장, API 연동, 데이터 처리** 등 다양한 응용 분야에서 중요한 역할을 합니다. 

In [28]:
# 사용자 정의 함수 또는 외부 API
def get_weather(location: str, unit: str = "섭씨"):
    return f"{location}의 날씨는 {unit} 20°."

In [27]:
# 스키마 정의
schema = {
    "name": "get_weather",   # 함수 이름 (고유 식별자 역할)
    "description": "지정된 위치의 현재 날씨를 조회합니다.",   # 함수에 대한 설명
    "parameters": {          # 함수가 받을 매개변수(parameters) 정의
        "type": "object",    # 매개변수 타입 (객체 형태로 입력받음)
        "properties": {      # 매개변수의 세부 속성 정의
            "location": {
                "type": "string",   # 입력 타입: 문자열
                "description": "날씨를 조회할 도시의 이름입니다."  # 매개변수 설명
            },
            "unit": {
                "type": "string",    # 입력 타입: 문자열
                "enum": ["celsius", "fahrenheit"],    # 허용 가능한 값 (섭씨 또는 화씨)
                "description": "온도 단위로, 섭씨(celsius) 또는 화씨(fahrenheit)입니다."
            }
        },
        "required": ["location"]     # 필수 매개변수 (location은 반드시 입력되어야 함)
    }
}

# 스키마를 모델에 바인딩
model_with_structure = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(schema)

# 사용자 입력
user_input = "서울의 현재 날씨를 알려줘."

# 사용자 입력을 받아 스키마에 맞는 구조화된 출력을 생성하도록 모델 호출
structured_output = model_with_structure.invoke(user_input)
structured_output

{'location': '서울'}

In [29]:
# structured_output을 함수 파라미터로 전달
response = get_weather(**structured_output)
print(response)

서울의 날씨는 섭씨 20°.


### **스키마를 도구로서 모델에 바인딩**

In [34]:
from pydantic import BaseModel, Field

# 사용자에게 응답을 구조화하기 위한 도구 정의
class ResponseFormatter(BaseModel):
    """항상 이 도구를 사용하여 사용자에게 응답을 구조화하세요."""
    
    answer: str = Field(
        description="사용자의 질문에 대한 답변"  # 필드 설명: 사용자 질문에 대한 답변 제공
    )
    followup_question: str = Field(
        description="사용자가 추가로 할 수 있는 후속 질문"  # 필드 설명: 사용자가 추가로 질문할 수 있는 내용 제공
    )

In [61]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  

# 스키마를 도구로 모델에 바인딩하여, 스키마를 따르는 출력을 생성하도록 설정  
model_with_tools = model.bind_tools([ResponseFormatter])  

# 모델 호출
ai_msg = model_with_tools.invoke("세포의 발전소는 무엇인가요??")  

ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_knLHD4OQDmqfO6VPNDUxPFP6', 'function': {'arguments': '{"answer":"세포의 발전소는 미토콘드리아입니다. 미토콘드리아는 세포 내에서 에너지를 생성하는 역할을 하며, 주로 ATP(아데노신 삼인산)를 생산하여 세포의 에너지 요구를 충족시킵니다. 이 과정은 호흡이라고 불리며, 산소를 사용하여 영양소를 분해하여 에너지를 생성합니다.","followup_question":"미토콘드리아의 기능에 대해 더 알고 싶으신가요?"}', 'name': 'ResponseFormatter'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 89, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-18d72a18-f6ca-40c8-8e73-eef17d700eec-0', tool_calls=[{'name': 'ResponseFormatter', 'args': {'answer': '세포의 발전소는 미토콘드리아입니다. 미토콘드리아는 세포 내에서 에너지를 생성하는 역할을 하

In [73]:
json_string = ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments']
json_string

'{"answer":"세포의 발전소는 미토콘드리아입니다. 미토콘드리아는 세포 내에서 에너지를 생성하는 역할을 하며, 주로 ATP(아데노신 삼인산)를 생산하여 세포의 에너지 요구를 충족시킵니다. 이 과정은 호흡이라고 불리며, 산소를 사용하여 영양소를 분해하여 에너지를 생성합니다.","followup_question":"미토콘드리아의 기능에 대해 더 알고 싶으신가요?"}'

In [74]:
import json
json.loads(json_string)

{'answer': '세포의 발전소는 미토콘드리아입니다. 미토콘드리아는 세포 내에서 에너지를 생성하는 역할을 하며, 주로 ATP(아데노신 삼인산)를 생산하여 세포의 에너지 요구를 충족시킵니다. 이 과정은 호흡이라고 불리며, 산소를 사용하여 영양소를 분해하여 에너지를 생성합니다.',
 'followup_question': '미토콘드리아의 기능에 대해 더 알고 싶으신가요?'}